# advent of code 2024 - [day 6](https://adventofcode.com/2024/day/6)

In [130]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [131]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

2.13.0


In [132]:
queries = [
"CALL apoc.schema.assert({},{});",
"""MATCH (n)
CALL {WITH n DETACH DELETE n}
IN TRANSACTIONS OF 1000 ROWS;""",
"""CALL gds.graph.list()
YIELD graphName
WITH graphName AS g
CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
WITH graphName RETURN graphName;"""]

for q in queries:
    gds.run_cypher(q, {})


In [133]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line.strip()):
            yield ix, jx, c

In [134]:
filename = "test.txt"
#filename = "input.txt"

## Neo4j-based solution

### Parsing

In [135]:
tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists(filename))]

### Building Lattice

In [136]:
gds.run_cypher('CREATE INDEX tile_row IF NOT EXISTS FOR (r:Tile) ON (r.row)')
gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col)')
gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (r:Tile) ON (r.val)')
gds.run_cypher('CREATE INDEX port_comp IF NOT EXISTS FOR (p:Port) ON (p.row, p.col, p.dir)')
gds.run_cypher('CREATE INDEX has_port_dir FOR ()-[r:HAS_PORT]-() ON (r.dir)')


""


In [137]:
query_ingest = """
UNWIND $tiles AS tile
CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
"""

gds.run_cypher(query_ingest, {"tiles":tiles})

""


In [138]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.row AS row_num, t ORDER BY t.col
WITH row_num, collect(t) AS row
CALL apoc.nodes.link(row, 'EAST')
""")

""


In [139]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.col AS col_num, t ORDER BY t.row
WITH col_num, collect(t) AS col
CALL apoc.nodes.link(col, 'SOUTH')
""")

""


In [140]:
gds.run_cypher("""
MATCH (x)-[:EAST]->(y)
CALL (x,y) {
MERGE (x)-[:NEXT_TO {dir: '>'}]->(y)
MERGE (y)-[:NEXT_TO {dir: '<'}]->(x)
} IN TRANSACTIONS OF 1000 ROWS
""")

gds.run_cypher("""
MATCH (x)-[:SOUTH]->(y)
CALL (x,y) {
MERGE (x)-[:NEXT_TO {dir: 'v'}]->(y)
MERGE (y)-[:NEXT_TO {dir: '^'}]->(x)
} IN TRANSACTIONS OF 1000 ROWS
""")

""


In [141]:
gds.run_cypher("""
MATCH (t:Tile)
CALL (t){
UNWIND ['v','<','^','>'] AS dir
MERGE (p:Port{row:t.row, col:t.col, dir:dir})
MERGE  (t)-[:HAS_PORT {dir:dir}]->(p)
} IN CONCURRENT TRANSACTIONS OF 1000 ROWS
""")

""


In [142]:
gds.run_cypher("""
MATCH (t1:Tile)-[r:NEXT_TO]->(t2:Tile)
WHERE t2.val IN ['.', 'v', '<', '^', '>'] AND t1.val IN ['.', 'v', '<', '^', '>']
CALL (t1, r, t2) {
MATCH (t1)-[:HAS_PORT {dir: r.dir}]->(p1), (t2)-[:HAS_PORT {dir: r.dir}]->(p2)
MERGE (p1)-[:POSSIBLE_MOVE]->(p2)
} IN TRANSACTIONS OF 100 ROWS
""")

gds.run_cypher("""
MATCH (t:Tile)-[r1:NEXT_TO]->(t1:Tile)
WHERE t.val <> '#' AND t1.val = '#'
CALL (t, r1) {
MATCH (t)-[:HAS_PORT {dir: r1.dir}]->(p1), (t)-[:HAS_PORT {dir: {`v`:'<', `<`:'^', `^`:'>', `>`:'v'}[r1.dir]}]->(p2)
MERGE (p1)-[:POSSIBLE_MOVE]->(p2)
} IN TRANSACTIONS OF 100 ROWS
""")

""


### Solving part1

In [143]:
gds.run_cypher("""
MATCH (start:Tile WHERE NOT start.val IN ['.','#'])-[:HAS_PORT {dir:start.val}]->(start_p)
MATCH path = (start_p)-[:POSSIBLE_MOVE]->*(x)
WITH DISTINCT x
MATCH (x)<-[:HAS_PORT]-(reachable_tile)
RETURN count(DISTINCT reachable_tile) AS part1
""")

,part1
0,41


### Adding obstruction links

In [144]:
gds.run_cypher("""
MATCH (t:Tile)-[r1:NEXT_TO]->(t1:Tile)
WHERE t.val <> '#' AND t1.val = '.'
CALL (t, r1, t1) {
MATCH (t)-[:HAS_PORT {dir: r1.dir}]->(p1), (t)-[:HAS_PORT {dir: {`v`:'<', `<`:'^', `^`:'>', `>`:'v'}[r1.dir]}]->(p2)
MERGE (p1)-[:OBSTRUCTED_MOVE {obstructing_tile: elementId(t1)}]->(p2)
} IN TRANSACTIONS OF 100 ROWS
""")

""
